In [132]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

import chardet


In [133]:
#find r0695 file in directory
keyword = 'R0695'
for fname in os.listdir('import_data'):
    if keyword in fname:
        filename = fname

#extract the dateof the file
date_list = filename [6:len(filename)-4].split('_')
r0695_date=date_list[0] + '-' + date_list [1] + '-' + date_list [2]


In [134]:
#read and clean up data R0695
r0695 = pd.read_csv('import_data/' + filename)
#save data to backup folder
r0695.to_csv(f'backup_data/{filename}',index=False)
r0695=r0695[['ItemId','VendorCode','VendorName','SOH','CostPerUnit','ItemGroup',
       'CoverageGroup','Brand','BuyerCode','SuperGrpDesc','AvgMthUsage']]
r0695.fillna('0',inplace=True)
#convert vendor code to int64
r0695['VendorCode'] = r0695['VendorCode'].astype('int64')

r0695['SOH'] = r0695['SOH'].astype(str) #to be confirm
r0695['CostPerUnit'] = r0695['CostPerUnit'].astype(str) #to be confirm

#convert SOH and Costperunit to float
r0695['SOH'] =r0695['SOH'].str.replace('(','-')
r0695['SOH'] =r0695['SOH'].str.replace(')','')
r0695['SOH'] = r0695['SOH'].str.replace(",","").astype(float)
r0695['CostPerUnit'] =r0695['CostPerUnit'].str.replace('(','-')
r0695['CostPerUnit'] =r0695['CostPerUnit'].str.replace(')','')
r0695['CostPerUnit'] = r0695['CostPerUnit'].str.replace(",","").astype(float)
#append date of data as String
r0695['Date'] = r0695_date

#open database, if not found file, generate data using r0695 data
try:
       dataset = pd.read_csv('archive/consolidate_data.csv')
except FileNotFoundError:
       dataset = r0695.copy()


#check if the r0695 data has duplicate date
duplicate_data = 0
#get list of date in database
dataset_date = dataset['Date'].unique().tolist()
for i in dataset_date:
       if r0695_date == i:
              #set flag if the date of r0695 is duplicated
              duplicate_data = 1  
              break

# if duplicate date found
if duplicate_data ==1:
       print('Duplicate value found')
       answer = input ('Do you want to override? (Y/N) :')
       if answer == 'Y':
              #remove old data for the duplicate date and itemid
              r0695_itemlist = r0695['ItemId'].tolist()
              dataset = dataset[(dataset["Date"]!=r0695_date) |  (~dataset['ItemId'].isin(r0695_itemlist)) ]    
              #append new data
              new_dataset = pd.concat([dataset,r0695],ignore_index=True)
              new_dataset.to_csv('archive/consolidate_data.csv',index=False)
       else:
              new_dataset = pd.read_csv('archive/consolidate_data.csv')
      
# if no duplicate date found, append new data
else:
       new_dataset = pd.concat([dataset,r0695],ignore_index=True)
       new_dataset.to_csv('archive/consolidate_data.csv',index=False)

#delete r0695 file after copy data
os.remove('import_data/'+filename)

#update latest standard cost
latest_std_cost = r0695[['ItemId','CostPerUnit','Date']]
latest_std_cost.rename(columns={'CostPerUnit':'Std_cost_latest'},inplace=True)
latest_std_cost.fillna(0,inplace=True)
latest_std_cost.to_csv('archive/latest_std_cost.csv',index=False)

new_dataset


c:\Users\datqu\anaconda3\envs\bootcamp\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  from ipykernel import kernelapp as app
c:\Users\datqu\anaconda3\envs\bootcamp\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  app.launch_new_instance()
c:\Users\datqu\anaconda3\envs\bootcamp\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
c:\Users\datqu\anaconda3\envs\bootcamp\lib\site-packages\ipykernel_launcher.

,ItemId,VendorCode,VendorName,SOH,CostPerUnit,ItemGroup,CoverageGroup,Brand,BuyerCode,SuperGrpDesc,AvgMthUsage,Date
0,001/T1012KITCP,0,0,0.0,13.2245,FG_MAN,MTO,LOCKWOOD,0,70 - 70 Mechanical Keying,0,15-06-2022
1,001/T1012KITSC,0,0,0.0,11.9497,FG_MAN,MTO,LOCKWOOD,0,70 - 70 Mechanical Keying,3.33,15-06-2022
2,001/T1023KITSC,0,0,0.0,11.9520,FG_MAN,MTO,LOCKWOOD,0,70 - 70 Mechanical Keying,3.33,15-06-2022
3,001/T1024KITSC,0,0,0.0,11.9520,FG_MAN,MTO,LOCKWOOD,0,70 - 70 Mechanical Keying,0,15-06-2022
4,001/T1034KITSC,0,0,0.0,11.9520,FG_MAN,MTO,LOCKWOOD,0,70 - 70 Mechanical Keying,1.67,15-06-2022
...,...,...,...,...,...,...,...,...,...,...,...,...
257900,ZT692-15,0,0,0.0,0.3399,COM_MAN,PHANTOM,NONSALES,0,99 - 99 Non Sales Reporting Group,0,06-07-2022
257901,ZT811-70,71227,Diecast Australia Pty Ltd,0.0,0.7700,COM_PUREXT,PTS_RM_WK,NONSALES,L3,99 - 99 Non Sales Reporting Group,0,06-07-2022
257902,ZT8474-20,0,0,0.0,0.8458,COM_MAN,PHANTOM,NONSALES,0,99 - 99 Non Sales Reporting Group,0,06-07-2022
257903,ZZ51081001,31773,Hempel (Wattyl) Australia Pty Ltd,0.0,0.0000,Consumable,UNPLANNED,0,0,99 - 99 Non Sales Reporting Group,0,06-07-2022


In [135]:
#merge latest data to database and calculate stock value
new_dataset = pd.read_csv('archive/consolidate_data.csv')
latest_std_cost = pd.read_csv('archive/latest_std_cost.csv')

del latest_std_cost['Date']
#append latest standard cost to database
process_dataset=new_dataset.merge(latest_std_cost,on='ItemId',how='left')
#check if any latest_std_cost is missing. If yes fill with original CostPerUnit data
process_dataset['Std_cost_latest'].fillna(process_dataset['CostPerUnit'],inplace=True)
#calculate stock value = soh * standard cost
process_dataset ['Stock_value'] = process_dataset['SOH'] * process_dataset['Std_cost_latest']
process_dataset['VendorCode'] =process_dataset['VendorCode'].astype('str')
process_dataset['Vendor'] = process_dataset['VendorCode'] + ' - ' + process_dataset['VendorName']
process_dataset['CoverageGroup1'] = process_dataset['ItemGroup']
process_dataset['CoverageGroup1'].replace({'BYPROD':'Other','0':'Other','Consumable':'Other','EngUse':'Other','FacSup':'Other',
                                           'FrghtChg':'Other','InstlChg':'Other','LabrChg':'Other','MOC':'Other','NBNInstal':'Other',
                                           'SPECKEYCHG':'Other','TOOL':'Other','MK_STORE':'Other'},inplace=True)
'''
process_dataset['CoverageGroup1'].replace({'PTS_WK':'PUR_FG','PTS_2WK':'PUR_FG','PTS_MTH':'PUR_FG','PTS_3MTH':'PUR_FG',
                                           'PTO_CUS':'PUR_FG','PTO_NAF':'PUR_FG',                                                                                                                           
                                           'PTS_RM_WK':'PUR_RM','PTS_RM_2WK':'PUR_RM','PTS_RM_MTH':'PUR_RM','PTS_KANBAN':'PUR_RM',                                          
                                           'PTO_RM_NAF':'PUR_RM',
                                           'MTS':'MAN_FG','MTO':'MAN_FG','KANBAN':'MAN_FG','SPK':'MAN_FG','SPK_MTS':'MAN_FG',                                                                             
                                           'PHANTOM':'Other','0':'Other','ROS':'Other'
                                           },inplace=True)
'''
latest_std_cost.head()


,ItemId,Std_cost_latest
0,001/T1012KITCP,13.2245
1,001/T1012KITSC,11.9497
2,001/T1023KITSC,11.9520
3,001/T1024KITSC,11.9520
4,001/T1034KITSC,11.9520


In [136]:
#aggregate stock value by supplier and date

category = 'Vendor'
process_dataset.groupby([category,'Date']).sum()['Stock_value'].reset_index().to_csv('archive/supplier_stock_value.csv',index=False)

In [137]:

category = 'Vendor'
process_dataset.loc[process_dataset['ItemGroup'].isin(['FG_PURIG',
                                                       'FG_PUREXT']),:].groupby([category,'Date']).sum()['Stock_value'].reset_index().to_csv('archive/supplier_stock_value_FG.csv',index=False)

In [138]:
#aggregate by SuperGrpDesc
category = 'SuperGrpDesc'
process_dataset.groupby([category,'Date']).sum()['Stock_value'].reset_index().to_csv('archive/supergroup_stock_value.csv',index=False)

In [139]:
#aggregate by item group
category = 'ItemGroup'
process_dataset.groupby([category,'Date']).sum()['Stock_value'].reset_index().to_csv('archive/itemgroup_stock_value.csv',index=False)

In [140]:
#aggregate by coverage group

category = 'CoverageGroup1'
process_dataset.groupby([category,'Date']).sum()['Stock_value'].reset_index().to_csv('archive/coveragegroup_stock_value.csv',index=False)

In [141]:
#aggregate summary

process_dataset.groupby(['SuperGrpDesc','BuyerCode','CoverageGroup1','Vendor','Date']).sum()['Stock_value'].reset_index().to_csv('archive/summary_aggregate.csv',index=False)

In [142]:
#aggregate summary

curdate_string = [r0695_date]
#filter latest week data
item_summary = process_dataset[process_dataset['Date'].isin(curdate_string)]
item_summary.loc[item_summary.ItemId=='W286217',:]

,ItemId,VendorCode,VendorName,SOH,CostPerUnit,ItemGroup,CoverageGroup,Brand,BuyerCode,SuperGrpDesc,AvgMthUsage,Date,Std_cost_latest,Stock_value,Vendor,CoverageGroup1
256146,W286217,0,0,900.0,0.6949,FG_MAN,MTO,WHITCO,0,10 - 10 Window Lock/Access,41.67,06-07-2022,0.6949,625.41,0 - 0,FG_MAN


In [150]:
#load R0831
forecast = pd.read_csv('import_data/R0831.csv',header=2)
forecast = forecast[['ItemId1','Std_Cost',
                         'FC_CUR1','Forecast_Current_plus_1','Forecast_Current_plus_2','Forecast_Current_plus_3','Forecast_Current_plus_4',
                         'Forecast_Current_plus_5','Forecast_Current_plus_6','ReqGroupId1','Usage_3_Months']]
#exclude PTO
forecast = forecast.loc[(forecast['ReqGroupId1']!='PTO_CUS')&(forecast['ReqGroupId1']!='PTO_NAF')&(forecast['ReqGroupId1']!='PTO_RM_NAF'),:]
forecast.rename(columns={'ItemId1':'ItemId'},inplace = True)

forecast=forecast.fillna(0)
forecast['Std_Cost'] = forecast['Std_Cost'].astype(str) #to be confirm
forecast['Std_Cost']= forecast['Std_Cost'].str.replace('(','-')
forecast['Std_Cost'] = forecast['Std_Cost'].str.replace(')','')
forecast['Std_Cost'] =  forecast['Std_Cost'].str.replace(",","").astype(float)
forecast['FC_CUR1'] = forecast['FC_CUR1'].astype(str) #to be confirm
forecast['FC_CUR1']= forecast['FC_CUR1'].str.replace('(','-')
forecast['FC_CUR1'] = forecast['FC_CUR1'].str.replace(')','')
forecast['FC_CUR1'] =  forecast['FC_CUR1'].str.replace(",","").astype(float)
forecast['Usage_3_Months'] = forecast['Usage_3_Months'].astype(str) #to be confirm
forecast['Usage_3_Months']= forecast['Usage_3_Months'].str.replace('(','-')
forecast['Usage_3_Months'] = forecast['Usage_3_Months'].str.replace(')','')
forecast['Usage_3_Months'] =  forecast['Usage_3_Months'].str.replace(",","").astype(float)
forecast['curFC_cost'] = forecast.Std_Cost * forecast.FC_CUR1
for i in range (1,7):
    forecast[f'Forecast_Current_plus_{i}'] = forecast[f'Forecast_Current_plus_{i}'].astype(str) #to be confirm
    forecast[f'Forecast_Current_plus_{i}']= forecast[f'Forecast_Current_plus_{i}'].str.replace('(','-')
    forecast[f'Forecast_Current_plus_{i}'] = forecast[f'Forecast_Current_plus_{i}'].str.replace(')','')
    forecast[f'Forecast_Current_plus_{i}'] =  forecast[f'Forecast_Current_plus_{i}'].str.replace(",","").astype(float)
    forecast[f'Forecast_Current_plus_{i}_cost'] = forecast.Std_Cost * forecast[f'Forecast_Current_plus_{i}']

#added usage 3 month and forecast 3 month value to process_dataset
forecast['Forecast_3_months'] = (forecast['Forecast_Current_plus_1']+ forecast['Forecast_Current_plus_2'] + forecast['Forecast_Current_plus_3'])/3
forecast_temp = forecast.copy()
forecast_temp = forecast_temp[['ItemId','Usage_3_Months','Forecast_3_months']]
#calculate usage per month
forecast_temp['Usage_3_Months']=forecast_temp['Usage_3_Months']/3
process_dataset=process_dataset.merge(forecast_temp,on='ItemId',how='left')
#fill with average mth usage if na
process_dataset['AvgMthUsage'] = process_dataset['AvgMthUsage'].astype(str) #to be confirm
process_dataset['AvgMthUsage']= process_dataset['AvgMthUsage'].str.replace('(','-')
process_dataset['AvgMthUsage'] = process_dataset['AvgMthUsage'].str.replace(')','')
process_dataset['AvgMthUsage'] =  process_dataset['AvgMthUsage'].str.replace(",","").astype(float)
process_dataset['Usage_3_Months'].fillna(process_dataset['AvgMthUsage'],inplace=True)
process_dataset['Forecast_3_months'].fillna(process_dataset['AvgMthUsage'],inplace=True)
process_dataset['Usage_3_Months_value']= process_dataset['Usage_3_Months'] * process_dataset['Std_cost_latest']
process_dataset['Forecast_3_months_value']= process_dataset['Forecast_3_months'] * process_dataset['Std_cost_latest']
del process_dataset['Usage_3_Months']
del process_dataset['Forecast_3_months'] 

#create slow moving data copy
slow_moving=forecast.copy()
#aggregate total data

forecast = forecast.groupby('ItemId').sum()[['curFC_cost','Forecast_Current_plus_1_cost','Forecast_Current_plus_2_cost','Forecast_Current_plus_3_cost',
                                 'Forecast_Current_plus_4_cost']].reset_index()
#normalize the dataframe                                 
forecast_column = ['curFC_cost','Forecast_Current_plus_1_cost','Forecast_Current_plus_2_cost','Forecast_Current_plus_3_cost',
                                 'Forecast_Current_plus_4_cost']
forecast = forecast.melt(id_vars='ItemId',value_vars=forecast_column,var_name='forecast_period',value_name='forecast_cost')

#create list of month curFC,CurFC+1... 
for i in range(3):
    date_list[i] = int(date_list[i])
month_list = [i if i<=12 else i-12 for i in range(date_list[1],date_list[1]+5)]
#calculate remaining date for current month
days_of_month_dict = [0,31,28,31,30,31,30,31,31,30,31,30,31]
remaining_days = days_of_month_dict[date_list[1]] - date_list[0]
#calculate total weeks has forcast data
total_weeks=remaining_days
for i in range(1,5):
    total_weeks += days_of_month_dict[month_list[i]]
total_weeks = int(total_weeks/7)
#replace forecast month by value
forecast.replace(['curFC_cost','Forecast_Current_plus_1_cost','Forecast_Current_plus_2_cost',
                  'Forecast_Current_plus_3_cost','Forecast_Current_plus_4_cost'],month_list,inplace=True)
forecast.forecast_period=forecast.forecast_period.astype(int)
#calculate forecast per week
forecast['forecast_per_week'] = forecast.apply(lambda row: round(row['forecast_cost'] * 7/ days_of_month_dict[row['forecast_period']],1),axis=1)


#create new forecsat data frame for each week
item_list=item_summary.ItemId.unique()
week_id = np.arange(0,total_weeks+1)
#convert string to datetime
curdate_datetime =datetime.strptime(r0695_date,'%d-%m-%Y')

#create base df
future_dates=[]
for i in range(0,total_weeks+1):
    future_dates.append (curdate_datetime+timedelta(days=7*i))
week_id_df= pd.DataFrame({'week_id':week_id,
                          'Date':future_dates})
week_id_df['day_of_month'] = pd.DatetimeIndex(week_id_df['Date']).day
week_id_df['forecast_period'] = pd.DatetimeIndex(week_id_df['Date']).month
week_id_df['last_date_of_month'] = week_id_df.apply(lambda x: days_of_month_dict[x['forecast_period']],axis=1)
week_id_df['days_to_end_of_month'] = week_id_df.last_date_of_month - week_id_df.day_of_month
def week_order (x):
    if x['days_to_end_of_month'] < 7:
        return 'Last'
    elif x['day_of_month'] <= 7:
        return 'First'
    else:
        return 'Mid'
week_id_df['week_order'] = week_id_df.apply(lambda x: week_order(x),axis=1)


# add base df with each item and concat to new df
def generate_vendor_df (df,item_list):
    list_dfs = []
    for i in item_list:
        df1= pd.DataFrame.copy(df)
        df1['ItemId']=i
        list_dfs.append(df1)
    return pd.concat(list_dfs)
forecast_process = generate_vendor_df(week_id_df,item_list)

#mege forecast data
forecast_process=forecast_process.merge(forecast,how='left',on=['ItemId','forecast_period'])
forecast_process.fillna(0,inplace=True)
forecast_process['Date'] = forecast_process['Date'].dt.strftime('%d-%m-%Y')
#calculate week forecast processed
forecast_process['remaining_cost_of_month'] = forecast_process.apply(lambda x: x['forecast_per_week'] * x['days_to_end_of_month'] /7 
                                                                     if ((x['days_to_end_of_month'] <7)&(x['days_to_end_of_month']>0)) else 0,axis=1)
#calculate remaining_cost_previous_month with current date =0
forecast_process['remaining_cost_previous_month'] = forecast_process.remaining_cost_of_month.shift(1)
forecast_process.loc[forecast_process.Date == r0695_date, 'remaining_cost_previous_month']  = 0

#current week = weekly forecast
forecast_process['forecast_per_week_processed']=forecast_process.forecast_per_week
#if first week: current week + last week
forecast_process.loc[forecast_process.week_order == 'First',
                     'forecast_per_week_processed'] = round(forecast_process.loc[forecast_process.week_order == 'First',
                                                                           'forecast_per_week'] * forecast_process.loc[forecast_process.week_order == 'First', 
                                                                                                                       'day_of_month']/7 + forecast_process.loc[forecast_process.week_order == 'First',
                                                                                                                                                                'remaining_cost_previous_month'],1)
#first date =0
forecast_process.loc[forecast_process.Date == r0695_date, 'forecast_per_week_processed']  = 0
# take essential data
forecast_process =forecast_process[['ItemId','Date','forecast_per_week_processed']]

forecast_process.head()

c:\Users\datqu\anaconda3\envs\bootcamp\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (79,94,96,97,103,108,113,118,122,123) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\Users\datqu\anaconda3\envs\bootcamp\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  if sys.path[0] == '':
c:\Users\datqu\anaconda3\envs\bootcamp\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  del sys.path[0]
c:\Users\datqu\anaconda3\envs\bootcamp\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: The defau

,ItemId,Date,forecast_per_week_processed
0,001/T1012KITCP,06-07-2022,0.0
1,001/T1012KITCP,13-07-2022,0.0
2,001/T1012KITCP,20-07-2022,0.0
3,001/T1012KITCP,27-07-2022,0.0
4,001/T1012KITCP,03-08-2022,0.0


In [144]:
#process open pos data

#check encoding
#with open('import_data/Open_POs.csv') as f:
#    print(f)

# look at the first ten thousand bytes to guess the character encoding
#with open("import_data/Open_POs.csv", 'rb') as rawdata:
#    result = chardet.detect(rawdata.read(10000))
# check what the character encoding might be
#print(result)

open_po = pd.read_csv('import_data/Open_POs.csv',encoding='utf8')
open_po = open_po[['Item number','Delivery date','Confirmed delivery date','Deliver remainder','Coverage group']]

#exclude PTO
open_po = open_po.loc[(open_po['Coverage group']!='PTO_CUS')&(open_po['Coverage group']!='PTO_NAF')&(open_po['Coverage group']!='PTO_RM_NAF'),:]
#fill confirmed delivery date = delivery date if empty
open_po['Confirmed delivery date'].fillna(open_po['Delivery date'],inplace=True)
open_po.fillna(0,inplace=True)
open_po['Deliver remainder'] = open_po['Deliver remainder'].astype(str) #to be confirm
open_po['Deliver remainder']= open_po['Deliver remainder'].str.replace('(','-')
open_po['Deliver remainder'] = open_po['Deliver remainder'].str.replace(')','')
open_po['Deliver remainder'] =  open_po['Deliver remainder'].str.replace(",","").astype(float)
#rename column to merge
open_po.rename(columns={'Item number':'ItemId'},inplace=True)
#merge with standard cost data
open_po=open_po.merge(latest_std_cost,on='ItemId',how='left') 
#calculate line item cost
open_po['po_cost'] = open_po['Deliver remainder'] * open_po.Std_cost_latest
open_po.fillna(0,inplace=True)

#convert confirmed delivery date to datetime
open_po['Confirmed delivery date'] = pd.to_datetime(open_po['Confirmed delivery date'],format='%d/%m/%Y')
#bucket the confirmed delivery date to week
#if confirmed delivery date <= before today +7 => today +7
#round to next today +7, return None if outside forecast period
def convert_po_date(df,future_dates_list):
    for i in future_dates_list:
        if df['Confirmed delivery date'] <= i:
            return i       
    return None
open_po['Date'] = open_po.apply(lambda x: convert_po_date(x,future_dates[1:len(future_dates)]), axis = 1)
#drop open PO outside forecast period
open_po.dropna(inplace=True)
#aggregate by vedor + new buket confirm delivery date
open_po = open_po.groupby(['ItemId','Date']).sum()['po_cost'].reset_index()
#convert date to string
open_po['Date'] = open_po['Date'].dt.strftime('%d-%m-%Y')
open_po.head()

c:\Users\datqu\anaconda3\envs\bootcamp\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
c:\Users\datqu\anaconda3\envs\bootcamp\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


,ItemId,Date,po_cost
0,001-1K1BRNDP,20-07-2022,914.88
1,001-1K1BRNDP,03-08-2022,571.80
2,001-1K1BRNDP,10-08-2022,343.08
3,001-1K1BRNDP,07-09-2022,914.88
4,001-1K1CP,03-08-2022,1108.80


In [145]:
#merge stock value df with forecast and open po data
merge_df = item_summary.merge(forecast_process,how='outer',on=['ItemId','Date'])
merge_df = merge_df.merge(open_po,how='outer',on=['ItemId','Date'])
#fill in empty value by = 0
merge_df['Stock_value'].fillna(0,inplace=True)
merge_df['forecast_per_week_processed'].fillna(0,inplace=True)
merge_df['po_cost'].fillna(0,inplace=True)
#calculate the intake value
merge_df['intake_cost'] = merge_df.Stock_value - merge_df.forecast_per_week_processed + merge_df.po_cost

#remove no SOH,noPO,no forecast item
no_value_filter = merge_df.groupby('ItemId').sum()[['Stock_value','forecast_per_week_processed','po_cost','intake_cost']].reset_index()
item_filter_list = no_value_filter.loc[(
    no_value_filter.Stock_value==0)&(no_value_filter.forecast_per_week_processed==0)&(no_value_filter.po_cost==0),'ItemId']
merge_df=merge_df.loc[~merge_df['ItemId'].isin(item_filter_list),:]
#get needed data only
merge_df =merge_df[['ItemId','Date','intake_cost']]

#Calculate EOH stock value
unique_items = merge_df['ItemId'].unique()
for i in unique_items:
    merge_df.loc[merge_df.ItemId == i,'eoh_value'] = merge_df.loc[merge_df.ItemId == i,'intake_cost'].cumsum()
merge_df.loc[merge_df.ItemId=='W286217',:]



,ItemId,Date,intake_cost,eoh_value
62743,W286217,06-07-2022,625.41,625.41
1382105,W286217,13-07-2022,0.00,625.41
1382106,W286217,20-07-2022,0.00,625.41
1382107,W286217,27-07-2022,0.00,625.41
1382108,W286217,03-08-2022,0.00,625.41
1382109,W286217,10-08-2022,0.00,625.41
1382110,W286217,17-08-2022,0.00,625.41
1382111,W286217,24-08-2022,0.00,625.41
1382112,W286217,31-08-2022,0.00,625.41
1382113,W286217,07-09-2022,0.00,625.41


In [152]:
#create final dataframe
projected_stock_value = merge_df.copy()
projected_stock_value = projected_stock_value [['ItemId','Date','eoh_value']]

#make format equal original dataframe format
projected_stock_value.rename(columns={'eoh_value':'Stock_value'},inplace=True)     
#exclude current date
projected_stock_value = projected_stock_value.loc[projected_stock_value['Date']!=r0695_date,:]
#create item attribute
item_attribute = process_dataset.loc[process_dataset.Date == r0695_date,['ItemId','ItemGroup','CoverageGroup','Brand','BuyerCode',
                                                                         'SuperGrpDesc',
                                                                         'Std_cost_latest','Vendor','CoverageGroup1']]
item_attribute.drop_duplicates(subset=['ItemId'],inplace=True)
#extract history of stock to merge
item_stock_hist = process_dataset[['ItemId','Date','Stock_value','Usage_3_Months_value','Forecast_3_months_value']]
#concate projected data to current stock data
projected_stock_value = pd.concat([item_stock_hist,projected_stock_value])
#merge item attribute to get full table
projected_stock_value = projected_stock_value.merge(item_attribute,on='ItemId',how='inner')
#add additional column to determine current date
projected_stock_value['Latest_data_date'] = r0695_date
#generate is_today,lastmonth,lastyear
last_month_date = curdate_datetime - timedelta(days=7*4)
last_year_date = curdate_datetime - timedelta(days=7*52)
last_month_date = last_month_date.strftime('%d-%m-%Y')
last_year_date = last_year_date.strftime('%d-%m-%Y')
projected_stock_value['is_today']='N'
projected_stock_value.loc[projected_stock_value.Date==r0695_date,'is_today'] ='today'
projected_stock_value.loc[projected_stock_value.Date==last_month_date,'is_today'] ='last_month_date'
projected_stock_value.loc[projected_stock_value.Date==last_year_date,'is_today'] ='last_year_date'

#flag slow moving stock if SOH >=7 month forcast or 7x month average usage
#sum forecast
slow_moving['fc'] = slow_moving.FC_CUR1 +slow_moving.Forecast_Current_plus_1+slow_moving.Forecast_Current_plus_2 +\
    slow_moving.Forecast_Current_plus_3+slow_moving.Forecast_Current_plus_4 +slow_moving.Forecast_Current_plus_5 +slow_moving.Forecast_Current_plus_6
k = slow_moving[['ItemId','fc']]
#merge
slow_moving_df = item_summary.merge(k,on='ItemId',how='left')
#convert to float
slow_moving_df['AvgMthUsage'] = slow_moving_df['AvgMthUsage'].astype(str) #to be confirm
slow_moving_df['AvgMthUsage']= slow_moving_df['AvgMthUsage'].str.replace('(','-')
slow_moving_df['AvgMthUsage'] = slow_moving_df['AvgMthUsage'].str.replace(')','')
slow_moving_df['AvgMthUsage'] =  slow_moving_df['AvgMthUsage'].str.replace(",","").astype(float)
#if no forecast us average usage

slow_moving_df['fc'].fillna(slow_moving_df['AvgMthUsage']*7,inplace=True)
slow_moving_df['slow_moving']=slow_moving_df.apply(lambda y: "Normal stock" if (y['SOH'] - y['fc']) <= 0 else "Slow moving stock",axis=1)
slow_moving_df = slow_moving_df[['ItemId','slow_moving']]
projected_stock_value =projected_stock_value.merge(slow_moving_df,on='ItemId',how='left')
projected_stock_value.fillna(0,inplace=True)
#save to csv
projected_stock_value.to_csv('archive/summary_aggregate_item_projected.csv',index=False)


c:\Users\datqu\anaconda3\envs\bootcamp\lib\site-packages\ipykernel_launcher.py:41: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
c:\Users\datqu\anaconda3\envs\bootcamp\lib\site-packages\ipykernel_launcher.py:42: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
